<a href="https://colab.research.google.com/github/LangeHeris/stable-diffusion-webui-colab/blob/main/%E7%99%B8%E5%8D%AF%E4%B8%99%E8%BE%B0%E5%A3%AC%E5%AF%85%E4%B8%99%E5%8D%88_%E8%81%9A%E7%81%B5%E9%98%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
from google.colab import drive
drive.mount('/content/drive')
import os

#环境匹配
!pip install torchvision==0.15.1
!pip install -U xformers
!apt -y install -qq aria2

#下载整合包，同步配置文件
!test -f /content/drive/MyDrive/SD-WebUI-Colab.zip || aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/langeheris/P-SD-Model/resolve/main/SD-WebUI-Colab.zip -o SD-WebUI-Colab.zip -d /content
!test -f /content/SD-WebUI-Colab.zip && unzip /content/SD-WebUI-Colab.zip -d /content || unzip /content/drive/MyDrive/SD-WebUI-Colab.zip -d /content
!test -f /content/drive/MyDrive/stable-diffusion-models/config.json && ln -sf /content/drive/MyDrive/stable-diffusion-models/config.json /content/stable-diffusion-webui-master/config.json
!rm SD-WebUI-Colab.zip

#@markdown #备份整合包到云盘，下次从云盘解压。
save = False #@param {type:"boolean"}
if save:
  !test -f /content/drive/MyDrive/SD-WebUI-Colab.zip || cp /content/SD-WebUI-Colab.zip /content/drive/MyDrive/

#@markdown #选择Lora路径，启用Additional-networks选择第二个。
Lora_PATH = "/content/stable-diffusion-webui-master/extensions/sd-webui-additional-networks/models/lora" #@param ["/content/stable-diffusion-webui-master/models/Lora","/content/stable-diffusion-webui-master/extensions/sd-webui-additional-networks/models/lora"]

#@markdown #复制云端模型
!test -d /content/drive/MyDrive/stable-diffusion-models/models || mkdir /content/drive/MyDrive/stable-diffusion-models/models -p
SD_PATH = "/content/drive/MyDrive/stable-diffusion-models/models/Stable-diffusion"
LORA_PATH = "/content/drive/MyDrive/stable-diffusion-models/models/Lora"
use_drive = True #@param {type:"boolean"}
if use_drive:

    drive_lora_list = [
        "adventurers_v1.safetensors",
        "Moxin_10.safetensors",
        "Moxin_Shukezouma11.safetensors",
        "Xiaorenshu_v20.safetensors",
        "pastelMixStylizedAnime_pastelMixLoraVersion.safetensors",
        "sunAndShadow_v10.safetensors",
        "tokiameStyleThick_1.safetensors",
    ]
    
    for dll in drive_lora_list:
        !cp -r {LORA_PATH}"/"{dll} {L_PATH}

#@markdown #使用共享模型
SHARE_SD_PATH = "/content/drive/MyDrive/share_stable-diffusion"
SHARE_LORA_PATH = "/content/drive/MyDrive/share_lora"
share = True #@param {type:"boolean"}
if share:
    share_stable_diffusion_list = [
        #"meinamix_meinaV8.safetensors",
        #"chilloutmix_NiPrunedFp32Fix.safetensors",
    ]
    share_lora_list = [
        "Colorwater_v4.safetensors",
    ]
    for ssdl in share_stable_diffusion_list:
        !cp -r "{SHARE_SD_PATH}/{ssdl}" /content/stable-diffusion-webui-master/models/Stable-diffusion
    for sll in share_lora_list:
        !cp -r {SHARE_LORA_PATH}"/"{sll} {L_PATH}

#@markdown #使用官方VAE
VAE = True #@param {type:"boolean"}
if VAE:
  !aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -o vae-ft-mse-840000-ema-pruned.safetensors -d /content/stable-diffusion-webui-master/models/VAE

#@markdown #下载额外扩展
E_PATH = "/content/stable-diffusion-webui-master/extensions"
extensions = True #@param {type:"boolean"}
if extensions:
  extension_names = [
    "https://github.com/etherealxx/batchlinks-webui",
    "https://github.com/Mikubill/sd-webui-controlnet",
    "https://github.com/camenduru/stable-diffusion-webui-huggingface",
    #"https://github.com/albarji/mixture-of-diffusers",
    "https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111"
  ]
  
  for url in extension_names:
    !git clone {url} {E_PATH}/{url.split("/")[-1]}

#@markdown #CodeFormer预下载
use_cf = True #@param {type:"boolean"}  
if use_cf: 
  !aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth -o detection_Resnet50_Final.pth -d /content/stable-diffusion-webui-master/repositories/CodeFormer/weights/facelib/detection_Resnet50_Final.pth
  !aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/parsing_parsenet.pth -o parsing_parsenet.pth -d /content/stable-diffusion-webui-master/repositories/CodeFormer/weights/facelib/parsing_parsenet.pth
  !aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth -o codeformer.pth -d /content/stable-diffusion-webui-master/models/Codeformer/codeformer-v0.1.0.pth

#@markdown #直接启动
start = False #@param {type:"boolean"}
if start:
  %cd /content/stable-diffusion-webui-master
  !python launch.py --share --xformers --enable-insecure-extension-access --theme dark --gradio-queue

【[AI词汇加速器](https://ai.dawnmark.cn/)】
【[Danbooru 标签超市](https://tags.novelai.dev/)】
【[Aigodlike社区](https://www.aigodlike.com/)】
【[Arthub](https://arthub.ai/)】
【[Civitai](https://civitai.com/)】

In [ ]:
#启动聚灵阵
%cd /content/stable-diffusion-webui-master
!python launch.py --share --xformers --enable-insecure-extension-access --theme dark --gradio-queue --no-half-vae

In [ ]:
#同步配置文件
!cp /content/stable-diffusion-webui-master/config.json /content/drive/MyDrive/stable-diffusion-models
!ln -sf /content/drive/MyDrive/stable-diffusion-models/config.json /content/stable-diffusion-webui-master/config.json

In [ ]:
#下载模型
P_SD_Model = "https://huggingface.co/langeheris/P-SD-Model/resolve/main/"
Stable_diffusion_list = [
#mistoonanime v1
#https://civitai.com/api/download/models/28861
"painting_v3/dalcefopaintingv3.safetensors",
#"mindplayer-floralboys/mindplayerFloralboys_mindplayerFloralboys.ckpt",
#"dalcefo_painting_v5/dalcefo_painting_v5.safetensors",
#"darkSushiMixMix/darkSushiMixMix_brighterPruned.safetensors",
#"darkSushiMixMix_brighter/darkSushiMixMix_brighter.safetensors",
#"anythingelseV4/anythingelseV4_v45.safetensors",
"tmndMixIII/tmndMix_tmndMixIII.safetensors",
#"blueboys2D_v2/blueboys2D_v20.safetensors",
"pastelboys2D_v2/pastelboys2D_v20.safetensors",
#"pastelboys2D_v3/pastelboys2D_v30.safetensors",
#"cetusMix_Coda2/cetusMix_Coda2.safetensors",
]
Lora_list = [
#2D Sprite style
"https://civitai.com/api/download/models/29226",    
#"pastelMixStylizedAnime_pastelMixLoraVersion.safetensors",
#"sunAndShadow_v10.safetensors",
#"adventurers_v1.safetensors",
#"Colorwater_v4.safetensors",
#"Moxin_Shukezouma11.safetensors",
#"lora-Moxin_10.safetensors",
#"lora-Xiaorenshu_v20.safetensors",
#"lora-tokiameStyleThick_1.safetensors",
]
ControlNet_list = [
#黑色背景上带有白色边缘的单色图像。
#"https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_canny.pth",
#灰色图像，黑色表示深区域，白色表示浅区域。
#"https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_depth.pth",
#黑色背景上带有白色柔和边缘的单色图像。
#"https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_hed.pth",
#仅由黑色背景上的白色直线组成的单色图像。
#"https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_mlsd.pth",
#法线映射图像。
#"https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_normal.pth",
#OpenPose 骨骼图像。
#"https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth",
#手绘单色图像，黑色背景上有白色轮廓。
#"https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_scribble.pth",
#ADE20K的分段协议图像。
#"https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_seg.pth"
]

S_PATH = "/content/stable-diffusion-webui-master/models/Stable-diffusion"
#选择Lora路径，启用Additional-networks选择第二个。
#L_PATH = "/content/stable-diffusion-webui-master/extensions/sd-webui-additional-networks/models/lora" #@param ["/content/stable-diffusion-webui-master/models/Lora","/content/stable-diffusion-webui-master/extensions/sd-webui-additional-networks/models/lora"]
C_PATH = "/content/stable-diffusion-webui-master/models/ControlNet"

for url in Stable_diffusion_list:
    if 'http' not in url:
        url = P_SD_Model + url
    if 'huggingface.co' in url:
        filename = url.split("/")[-1]
    else:
        filename = ""
        
    !aria2c --console-log-level=error -c -x 16 -k 1M -s 16 {url} {("-o " + filename) if filename else ""} -d {S_PATH}

for url in Lora_list:
    if 'http' not in url:
        url = P_SD_Model + url
    if 'huggingface.co' in url:
        filename = url.split("/")[-1]
    else:
        filename = ""  
    !aria2c --console-log-level=error -c -x 16 -k 1M -s 16 {url} {("-o " + filename) if filename else ""} -d {L_PATH}

for url in ControlNet_list:
    if 'huggingface.co' in url:
        filename = url.split("/")[-1]
    else:
        filename = ""
        
    !aria2c --console-log-level=error -c -x 16 -k 1M -s 16 {url} {("-o " + filename) if filename else ""} -d {C_PATH}

#@markdown 模型备份到云盘,确保有足够空间。

#@markdown #Stable-diffusion 备份到云盘,确保有足够空间。
M_S2D = False #@param {type:"boolean"}
if M_S2D:
  !test -f /content/drive/MyDrive/stable-diffusion-models/models/Stable-diffusion || mkdir /content/drive/MyDrive/stable-diffusion-models/models/Stable-diffusion
  !cp -r {S_PATH}/* /content/drive/MyDrive/stable-diffusion-models/models/Stable-diffusion
#@markdown #Lora 备份到云盘,确保有足够空间。 

L_S2D = True #@param {type:"boolean"}
if L_S2D:
  !test -f /content/drive/MyDrive/stable-diffusion-models/models/Lora || mkdir /content/drive/MyDrive/stable-diffusion-models/models/Lora
  !cp -r {L_PATH}/* /content/drive/MyDrive/stable-diffusion-models/models/Lora


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
bf18ec|OK  |       0B/s|/content/stable-diffusion-webui-master/models/Stable-diffusion/dalcefopaintingv3.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
a9becb|OK  |   235MiB/s|/content/stable-diffusion-webui-master/models/Stable-diffusion/tmndMix_tmndMixIII.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
2f0a61|OK  |   227MiB/s|/content/stable-diffusion-webui-master/models/Stable-diffusion/pastelboys2D_v20.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+======================================================